# 🎬 Video Trans Studio - AI 视频配音与口型同步
### 核心能力：
- **ASR**: Faster-Whisper (Large-v3)
- **Translation**: NLLB-200 (HuggingFace Local)
- **TTS**: Index-TTS2 (Zero-shot Voice Cloning)
- **LipSync**: Wav2Lip-GAN

> **运行环境建议**：Google Colab Tesla T4 (16GB VRAM)

In [8]:
# @title 🔍 0. 环境检查 (排查问题专用)
import sys
import os
import torch
import platform
import psutil
from IPython.display import HTML, display

def get_size(bytes, suffix="B"):
    factor = 1024
    for unit in ["", "K", "M", "G", "T", "P"]:
        if bytes < factor: return f"{bytes:.2f}{unit}{suffix}"
        bytes /= factor

print("="*50)
print("💻 系统硬件环境")
print("="*50)
print(f"操作系统: {platform.system()} {platform.release()}")
print(f"处理器: {platform.processor()}")
print(f"物理核心数: {psutil.cpu_count(logical=False)}")
print(f"逻辑核心数: {psutil.cpu_count(logical=True)}")
print(f"总内存: {get_size(psutil.virtual_memory().total)}")

print("\n" + "="*50)
print("🚀 显卡 (GPU) 环境")
print("="*50)
if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        prop = torch.cuda.get_device_properties(i)
        print(f"GPU {i}: {prop.name}")
        print(f"  - 显存总量: {prop.total_memory / 1024**3:.2f} GB")
        print(f"  - 计算能力: {prop.major}.{prop.minor}")
    print(f"CUDA 版本: {torch.version.cuda}")
    print(f"cuDNN 版本: {torch.backends.cudnn.version()}")
else:
    print("❌ 未检测到可用 GPU，请在 '修改' -> '笔记本设置' 中开启 T4 GPU 加速。")

print("\n" + "="*50)
print("🐍 软件开发环境")
print("="*50)
print(f"Python 版本: {sys.version}")
print(f"PyTorch 版本: {torch.__version__}")
print(f"当前路径: {os.getcwd()}")

!nvidia-smi

💻 系统硬件环境
操作系统: Linux 6.6.105+
处理器: x86_64
物理核心数: 1
逻辑核心数: 2
总内存: 12.67GB

🚀 显卡 (GPU) 环境
GPU 0: Tesla T4
  - 显存总量: 14.74 GB
  - 计算能力: 7.5
CUDA 版本: 12.6
cuDNN 版本: 91002

🐍 软件开发环境
Python 版本: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
PyTorch 版本: 2.9.0+cu126
当前路径: /content/video-trans-studio/video-trans-studio/video-trans-studio
Sat Jan 31 05:53:54 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+=============

In [9]:
# @title 🚀 1. 环境初始化 (大约需要 3-5 分钟)
import os

# 1. 克隆项目 (如果是在 Colab 直接运行)
if not os.path.exists('video-trans-studio'):
    !git clone https://github.com/infinite-gaming-studio/video-trans-studio.git
    %cd video-trans-studio
else:
    %cd video-trans-studio

# 2. 执行一键安装脚本
!bash setup_colab.sh

print("✅ 环境准备就绪！")

Cloning into 'video-trans-studio'...
remote: Enumerating objects: 29, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 29 (delta 10), reused 24 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (29/29), 12.20 KiB | 1.74 MiB/s, done.
Resolving deltas: 100% (10/10), done.
/content/video-trans-studio/video-trans-studio/video-trans-studio/video-trans-studio
🚀 Starting Video Trans Studio Colab Setup...
📦 Installing system dependencies (ffmpeg)...
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
🐍 Installing Python libraries...
📥 Cloning Wav2Lip repository...
📥 Cloning Index-TTS2 repository...
Error downloading object: examples/emo_hate.wav (89e6e7e): Smudge error: Error downloading examples/emo_hate.wav (89e6e7eee1a28303776e9cf43971e9505529bd0e669f5fcf47f4d1370f9187c4): batch resp

In [10]:
# @title 📂 2. 选择素材
import os
import glob
from pathlib import Path

# 核心逻辑：自动抓取服务器根目录下 sample_video 文件夹中最新上传的视频
# 在 Colab 中，根目录通常是 /content
colab_root = Path('/content')
sample_dir = colab_root / 'sample_video'

# 兼容性处理：如果不在 Colab 环境，尝试查找当前目录下的 sample_video
if not sample_dir.exists():
    sample_dir = Path('sample_video')

video_path = None

if sample_dir.exists():
    # 支持的视频格式
    valid_extensions = ['*.mp4', '*.mov', '*.avi', '*.mkv', '*.webm']
    video_files = []
    for ext in valid_extensions:
        video_files.extend(glob.glob(str(sample_dir / ext)))
    
    if video_files:
        # 按修改时间（mtime）排序，取最新的一个
        video_path = max(video_files, key=os.path.getmtime)
        print(f"✅ 成功定位服务器根目录素材: {video_path}")
        print(f"🕒 最新上传时间: {os.path.getmtime(video_path)}")
    else:
        print(f"❌ 在 {sample_dir} 文件夹中未找到任何视频文件。")
else:
    print(f"❌ 未找到目录: {sample_dir.absolute()}")
    print("💡 请确保在 Colab 左侧侧边栏的根目录（/content）下创建了 sample_video 文件夹。")

❌ 未找到 sample_video 文件夹，请确保已创建并放入视频。


In [11]:
# @title ⚙️ 3. 运行全自动流水线
target_language = "en" # @param ["zh-cn", "en", "es", "fr", "ja"]
use_local_translation = True # @param {type:"boolean"}

import torch
from main import run_pipeline

# 强制清理显存
torch.cuda.empty_cache()

# 运行主程序
run_pipeline(video_path, target_lang=target_language)

print("\n✨ 处理完成！请在 output 文件夹中查看结果。")

ImportError: cannot import name 'pipeline' from 'transformers' (/usr/local/lib/python3.12/dist-packages/transformers/__init__.py)

In [ ]:
# @title 📺 4. 预览结果
import os
from IPython.display import Video

output_dir = 'output'
final_videos = [f for f in os.listdir(output_dir) if f.startswith('final_')]
if final_videos:
    latest_video = os.path.join(output_dir, sorted(final_videos)[-1])
    print(f"展示最新生成的视频: {latest_video}")
    display(Video(latest_video, embed=True, width=600))
else:
    print("❌ 未找到生成的视频文件。")